# Calculate Solvent Entropy for a capped amino acid (Solvent)

## Load Data

1. Load data into MDAnalysis Universe

In [1]:
import MDAnalysis as mda
# set the working dir to the root of repo inorder to use these path
tprfile = "data/1AKI_prod_60.tpr"
trrfile = "data/1AKI_prod_60.trr"
u = mda.Universe(tprfile, trrfile)

In [2]:
import dask
import dask.multiprocessing
dask.config.set(scheduler='processes')
from dask.distributed import Client, progress
client = Client()
client

2022-07-28 18:22:02,409 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-au6fh2kr', purging
2022-07-28 18:22:02,409 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-024x0wbs', purging
2022-07-28 18:22:02,410 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-9xmuk268', purging
2022-07-28 18:22:02,410 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-76fptn9s', purging
2022-07-28 18:22:02,410 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-8r24ogky', purging
2022-07-28 18:22:02,410 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-whhtuqau', purging
2022-07-28 18:22:02,411 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-9xguvzox', purging

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 16,Total memory: 7.70 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39441,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 7.70 GiB
Comm: tcp://127.0.0.1:41523,Total threads: 4
Dashboard: http://127.0.0.1:45231/status,Memory: 1.92 GiB
Nanny: tcp://127.0.0.1:36329,


In [3]:
import sys
import logging
import gc

from collections import Counter
from datetime import datetime

from joblib import delayed

from CodeEntropy.poseidon.extractData.readFiles import populateTopology, getCoordsForces, getDistArray
# # Energy is not needed
# from CodeEntropy.poseidon.extractData.readFiles import populateEnergy, UAEnergyGroup
from CodeEntropy.poseidon.extractData.HBRAD import distCutoffNc, UALevelRAD, HBCalc
from CodeEntropy.poseidon.extractData.forceTorques import calculateFTMatrix
from CodeEntropy.poseidon.extractData.dihedrals import calculateDihedrals
from CodeEntropy.poseidon.extractData.nearestNonlike2 import getShellAssignment, moleculePositionRankingRAD 
from CodeEntropy.poseidon.extractData.outputFiles import moleculeObjectPopulation
from CodeEntropy.poseidon.extractData.mainClass import clearClass

from CodeEntropy.poseidon.analysis.populateClasses import classPopulation
from CodeEntropy.poseidon.analysis.EECalculation import processEE
from CodeEntropy.poseidon.analysis.helper import memoryInfo, weightingPopulation

from CodeEntropy.FunctionCollection.PoseidonHelper import frame_iteration

In [8]:
# def load(container, start=0, end=-1, 
#     step=1, pureAtomNum=1, cutShell=None, 
#     excludedResnames=None,
#     water='WAT', verbose=False, thread=4):
#     """
#     Warning!!!

#         This requires a lot of memory to run with very little performance gain.

#     This is a initialization function to collect information from a MDanalysis universe into a data container for analysis using POSEIDON.

#     Parameters
#     ----------
#     container : MDAnalyse.Universe
#         A Universe object will all topology, dihedrals,coordinates and force information Check ``Example/create_new_universe.py`` on how to create a universe from existing data.
#     start : int or None, Optional, default: 0
#         Frame id to start analysis.
#     end : int or None, Optional, default: -1
#         Frame id to end analysis.
#     step : int, Optional, default: 1
#         Steps between frame.
#     pureAtomNum : int, Optional, default: 1
#         Reference molecule resid for pure liquid.
#     cutShell : float or None, Optional, default: 1
#         Explicit cut off shell. Default to None which uses the relative angular distance (RAD) algorithm. See Jonathan Higham and Richard H. Henchman , "Locally adaptive method to define coordination shell", J. Chem. Phys. 145, 084108 (2016)
#     excludedResnames : list of str or None, Optional, default: None
#         List of resnames to exclude from nearest non alike analysis.
#     water : list of str, Optional, default: "WAT"
#         Resname for water molecules.
#     verbose : bool, Optional, default: False
#         Print out progress of each step.
#     thread : int, Optional, default: 4
#         How many multiprocess thread to spawn

#     """
container=u
start=0
end=5
step=1
pureAtomNum=1
cutShell=None
excludedResnames=None,
water='WAT'
verbose=True
thread=4
startTime = datetime.now()
print(startTime)
verbosePrint = print if verbose else lambda *a, **k: None

waterTuple = ('SOL', 'WAT', 'HOH', 'TIP3') #needed for pdb as top file 
if water != 'WAT':
    waterTuple = (water)

iterations = 0

all_data = []

populateTopology(container, all_data, waterTuple)
verbosePrint('TOPOLOGY')
verbosePrint(datetime.now() - startTime)
sys.stdout.flush()


resids = Counter([(i.resname) for i in all_data])
verbosePrint(resids.keys())
if len(resids.keys()) == 1:
    verbosePrint('Pure system with reference ID: %s' % (pureAtomNum))

if excludedResnames != None:
    verbosePrint('EXCLUDED RESNAMES: %s' % (excludedResnames))


dimensions = None
allMoleculeList = []

#fix end frame
if end == -1:
    end = len(container.trajectory)
sys.setrecursionlimit(3000000)
job_list = []
for frame in range(int(start), int(end), int(step)):
    job_list.append(dask.delayed(frame_iteration)(
        container, 
        all_data, 
        dimensions, 
        startTime, 
        verbosePrint, 
        waterTuple, 
        cutShell, 
        excludedResnames,
        frame))
    
print("prepare to execute")
data = dask.compute(job_list, scheduler="distributed")

print("Done Threading Work")
#print(data)
# data = []
# for frame in range(int(start), int(end), int(step)):
#     data.append(fi_partial(frame))

print(len(data))
for item in data[0]:
    allMoleculeList = moleculeObjectPopulation(item[0], 
            allMoleculeList, item[1], item[2])
# #writing file here 
# with gzip.GzipFile('moleculeListAll.obj', 'wb') as pickleFile:
#     pickle.dump((allMoleculeList), pickleFile, protocol=2)
#     pickleFile.close()

print(datetime.now() - startTime)




2022-07-28 18:19:46.761391
TOPOLOGY
0:00:10.816648
dict_keys(['LYS', 'VAL', 'PHE', 'GLY', 'ARG', 'CYS', 'GLU', 'LEU', 'ALA', 'MET', 'HIS', 'ASP', 'ASN', 'TYR', 'SER', 'TRP', 'THR', 'GLN', 'ILE', 'PRO', 'SOL', 'CL'])
EXCLUDED RESNAMES: None
prepare to execute
frame = 3
< Timestep 3 with unit cell dimensions [69.52697 69.52697 69.52697 90.      90.      90.     ] >
COORDS-FORCES
0:00:14.470321
frame = 2
< Timestep 2 with unit cell dimensions [69.61369 69.61369 69.61369 90.      90.      90.     ] >
COORDS-FORCES
0:00:14.534201
frame = 1
< Timestep 1 with unit cell dimensions [69.476456 69.476456 69.476456 90.       90.       90.      ] >
COORDS-FORCES
0:00:14.535861
DIH
0:00:14.804284
DIH
0:00:14.890316
DIH
0:00:14.894539
frame = 4
frame = 0
< Timestep 4 with unit cell dimensions [69.57063 69.57063 69.57063 90.      90.      90.     ] >
< Timestep 0 with unit cell dimensions [69.57307 69.57307 69.57307 90.      90.      90.     ] >
COORDS-FORCES
0:00:16.530269
DIH
0:00:17.063608
COORDS-F

/home/donald/miniconda3/envs/CEtrial/lib/python3.10/site-packages/CodeEntropy/poseidon/extractData/HBRAD.py:192: RuntimeWarning: invalid value encountered in arccos
  angle1 = np.arccos(cosine_angle)


RAD
0:00:35.219296
HB
0:00:37.813414
NEAREST NON-LIKE ASSIGNMENT
0:00:37.847995
PROX
0:00:37.884287
RAD
0:00:35.603050
HB
0:00:38.440154
NEAREST NON-LIKE ASSIGNMENT
0:00:38.473420
RAD
0:00:35.809416
HB
0:00:38.495989
PROX
0:00:38.509796
NEAREST NON-LIKE ASSIGNMENT
0:00:38.532525
PROX
0:00:38.569533
NEAREST ARRAYS
0:00:39.726922
FTMATRIX
0:00:43.661345
ORIENTS
0:00:43.729755
FTMATRIX
0:00:44.487481
ORIENTS
0:00:44.554535
FTMATRIX
0:00:44.695925
ORIENTS
0:00:44.765300
RAD
0:01:04.065172
RAD
0:01:10.980770
HB
0:01:11.554396
NEAREST NON-LIKE ASSIGNMENT
0:01:11.609858
PROX
0:01:11.668300
HB
0:01:15.105755
NEAREST NON-LIKE ASSIGNMENT
0:01:15.147424
PROX
0:01:15.187237
FTMATRIX
0:01:29.186166
ORIENTS
0:01:29.305053
FTMATRIX
0:01:30.530090
ORIENTS
0:01:30.984170


2022-07-28 18:21:21,340 - distributed.protocol.core - CRITICAL - Failed to deserialize
Traceback (most recent call last):
  File "/home/donald/miniconda3/envs/CEtrial/lib/python3.10/site-packages/distributed/protocol/core.py", line 158, in loads
    return msgpack.loads(
  File "msgpack/_unpacker.pyx", line 194, in msgpack._cmsgpack.unpackb
  File "/home/donald/miniconda3/envs/CEtrial/lib/python3.10/site-packages/distributed/protocol/core.py", line 138, in _decode_default
    return merge_and_deserialize(
  File "/home/donald/miniconda3/envs/CEtrial/lib/python3.10/site-packages/distributed/protocol/serialize.py", line 497, in merge_and_deserialize
    return deserialize(header, merged_frames, deserializers=deserializers)
  File "/home/donald/miniconda3/envs/CEtrial/lib/python3.10/site-packages/distributed/protocol/serialize.py", line 410, in deserialize
    deserialize(
  File "/home/donald/miniconda3/envs/CEtrial/lib/python3.10/site-packages/distributed/protocol/serialize.py", line 42

TypeError: Could not serialize object of type list
Traceback (most recent call last):
  File "/home/donald/miniconda3/envs/CEtrial/lib/python3.10/site-packages/distributed/protocol/pickle.py", line 46, in dumps
    result = pickle.dumps(x, **dump_kwargs)
RecursionError: maximum recursion depth exceeded while calling a Python object

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/donald/miniconda3/envs/CEtrial/lib/python3.10/site-packages/cloudpickle/cloudpickle_fast.py", line 602, in dump
    return Pickler.dump(self, obj)
RecursionError: maximum recursion depth exceeded

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/donald/miniconda3/envs/CEtrial/lib/python3.10/site-packages/distributed/protocol/serialize.py", line 347, in serialize
    header, frames = dumps(x, context=context) if wants_context else dumps(x)
  File "/home/donald/miniconda3/en

In [15]:
def run_analysis(allMoleculeList, 
        temperature=298.0, 
        entropyEnergy=True, 
        level_list=['moleculeLevel'], 
        solvent=None, water='WAT',
        verbose=False, weighting=None, 
        forceUnits='kJ'):
    """Perform analysis using poseidon

    Parameters
    -----------
    temperature : float, Optional, default: 298.0
        Temperature for system.
    entropyEnergy : Bool, Optional, default: True
        Run entropy and energy analysis.
    level_list : list of str, Optional, default: ['moleculeLevel']
        Choose and refine the level of analyiss: moleculeLevel, residLevel_resname, atomLevel, soluteContacts.
    solvent : str, Optional, default: None
        Resname for solvent.
    water : str or list of str, Optional, default: 'WAT'
        Resname for water.
    verbose : bool, Optional, default: False
        Print out progress of each step.
    weighting : str or None, Optional, default: None
        Get weighing for each frame if the simulation is biased.
    forceUnits : str, Optional, default: 'kJ'
        Units of forces, kJ or Kcal. 

    Returns
    ---------
        Tuple of DataFrame, for each level analysed:
            solventData: dataframe that contains the result for solvent
            soluteData: dataframe that contains the result for solute
            contactMatrix: dataframe that contains the contact matrix
    """

    verbosePrint = print if verbose else lambda *a, **k: None
    startTime = datetime.now()
    print(startTime)



    waterTuple = ['WAT', 'wat', 'SOL', 'H2O', 'h2o', 'WAT_O', 'TIP3']
    if water != 'WAT':
        waterTuple = [water]
    if solvent == None: ##when solvent is NOT water
        solvent = waterTuple




    print('\nsolvent: %s' % (solvent))
    print('\nwater: %s' % (waterTuple))
    print('\n1. Populate Dictionaries\n')

    count = 1
    totAtoms = 0
    list_len = None
    atom_nums = []
    atom_count = 0
    objectIteration = 0

    EEclass = None 
    EEclass_residLevel_resname = None
    EEclass_soluteContacts = None
    EEclass_atomLevel = None


    class_str_list = ['EEclass', 'EEclass_residLevel_resname', 
            'EEclass_soluteContacts', 'EEclass_atomLevel', 
            'totAtoms', 'atom_nums', 'atom_count', 'objectIteration']

    memoryInfo(verbosePrint)
    print(datetime.now() - startTime)
    sys.stdout.flush()
    if totAtoms != 0:
        count = 2

    weighting_info = None
    atomList = allMoleculeList
    list_len = len(atomList[0])
    totAtoms += len(atomList)
    if count == 1:
        for i in atomList:
            atom_num = i[2]
            if atom_num not in atom_nums:
                atom_nums.append(atom_num)
            else:
                break
        if weighting != None:
            weighting_info = weightingPopulation(weighting)

    EEclass, EEclass_residLevel_resname, \
            EEclass_soluteContacts, EEclass_atomLevel, \
            atom_count = \
                classPopulation(
            atomList, entropyEnergy, 
            level_list, count, 
            atom_count, len(atom_nums), 
            waterTuple, temperature, 
            solvent, EEclass, 
            EEclass_residLevel_resname, EEclass_soluteContacts, 
            EEclass_atomLevel, 
            weighting_info, verbosePrint)
    count += 1 #needed for class initiation
    gc.enable()
    memoryInfo(verbosePrint)
    print(datetime.now() - startTime)
    sys.stdout.flush()


    print(datetime.now() - startTime)
    memoryInfo(verbosePrint)
    sys.stdout.flush()


    try:
        totFrames = float(totAtoms) / float(len(atom_nums))
        print('\nTotal number of frames: %s' % (totFrames))
        print('Number of atoms in each frame: %s' % (len(atom_nums)))
        print('Number of variables in each list: %s' % (list_len))
        num_frames = totFrames
    except ZeroDivisionError:
        logging.error('No frames to analyse, please chose correct '\
                'path to .obj files')
        num_frames = None
        sys.exit()

    ###once all the classes have been populated, calculate properties
    #and output to files

    print('\n2. Process Dictionaries')
    result_list = dict()
    print(level_list)
    for level in level_list:

        print('---level:', level)
        EEclass2, DSclass2 = None, None

        if level in [None, 'moleculeLevel']:
            EEclass2 = EEclass
        if level in ['residLevel_resname']:
            EEclass2 = EEclass_residLevel_resname
        if level in ['soluteContacts']:
            EEclass2 = EEclass_soluteContacts
        if level in ['atomLevel']:
            EEclass2 = EEclass_atomLevel


        if entropyEnergy:
            name = 'EE'
            result = processEE(num_frames, totFrames, EEclass2, 
                    solvent, waterTuple, 
                    temperature, level, name, forceUnits, verbosePrint)
            result_list[level] = result
    # #'''
    # ##Save each class as an object so that we can continue populating
    # #in stages.
    # if len(paths) != 0:
    #     objectIteration += 1
    #     if pathClasses:
    #         print('\n3. Save Dictionaries')
    #         for Aclass in class_str_list:
    #             with gzip.GzipFile('%s.obj' % (Aclass), 'wb') as pickleFile:
    #                 pickle.dump((locals()[Aclass]), pickleFile, protocol=2)
    #                 pickleFile.close()
    #                 #locals()[Aclass] = None
    #         print('Number of objectIteration cycles saved: %s' % 
    #                 (objectIteration))
    #         print('Total atoms processed: %s' % (totAtoms))

    # #'''


    sys.stdout.flush()
    print('\n')
    print(datetime.now() - startTime)
    return result_list

run_analysis(allMoleculeList)

2022-07-28 18:06:33.882125

solvent: ['WAT', 'wat', 'SOL', 'H2O', 'h2o', 'WAT_O', 'TIP3']

water: ['WAT', 'wat', 'SOL', 'H2O', 'h2o', 'WAT_O', 'TIP3']

1. Populate Dictionaries

memory use: 1.929 GB
0:00:00.000914
memory use: 1.929 GB
0:00:01.406618
0:00:01.407275
memory use: 1.929 GB

Total number of frames: 20.0
Number of atoms in each frame: 916
Number of variables in each list: 19

2. Process Dictionaries
['moleculeLevel']
---level: moleculeLevel


0:00:01.600541


/home/donald/miniconda3/envs/CEtrial/lib/python3.10/site-packages/CodeEntropy/poseidon/analysis/EECalculation.py:859: RuntimeWarning: invalid value encountered in double_scalars
  w = w ** 0.5


{'moleculeLevel': {'solventData':    nearest assigned shell_num   variable       value count
  0      ACE      WAT       1.0  Sor_test2    4.489916   155
  1      ACE      WAT       1.0     Strans  140.062995   155
  2      ACE      WAT       1.0       Srot  113.115592   155
  3      ACE      WAT       1.0      count        7.75   155
  4      ARG      WAT       1.0  Sor_test2    4.389286   358
  5      ARG      WAT       1.0     Strans  139.597899   358
  6      ARG      WAT       1.0       Srot  113.456453   358
  7      ARG      WAT       1.0      count        17.9   358
  8      NME      WAT       1.0  Sor_test2    2.782703   117
  9      NME      WAT       1.0     Strans  139.409094   117
  10     NME      WAT       1.0       Srot   112.39722   117
  11     NME      WAT       1.0      count        5.85   117,
  'soluteData':    resName      variable       value count
  0      ACE     WM_Strans  123.289652    20
  1      ACE       WM_Srot  102.152818    20
  2      ACE  WM_UA_Stran

2. Load data into POSEIDON object

In [16]:
from CodeEntropy.ClassCollection.PoseidonClass import Poseidon
startTime = datetime.now()
poseidon_object = Poseidon(container=u, start=0, end=20)
print(datetime.now() - startTime)

2022-07-28 18:07:05.426259
frame = 0
< Timestep 0 with unit cell dimensions [30.67833  30.40052  29.490604 90.       90.       90.      ] >
0:00:02.403687
frame = 1
< Timestep 1 with unit cell dimensions [30.682182 30.404337 29.494312 90.       90.       90.      ] >
0:00:04.069732
frame = 2
< Timestep 2 with unit cell dimensions [30.713255 30.435127 29.524178 90.       90.       90.      ] >
0:00:05.803433
frame = 3
< Timestep 3 with unit cell dimensions [30.598316 30.321232 29.41369  90.       90.       90.      ] >
0:00:07.367259
frame = 4
< Timestep 4 with unit cell dimensions [30.618622 30.341354 29.43321  90.       90.       90.      ] >
0:00:08.980108
frame = 5
< Timestep 5 with unit cell dimensions [30.619095 30.341824 29.433666 90.       90.       90.      ] >
0:00:10.749856
frame = 6
< Timestep 6 with unit cell dimensions [30.738136 30.459784 29.5481   90.       90.       90.      ] >
0:00:12.403620
frame = 7
< Timestep 7 with unit cell dimensions [30.694881 30.416918 29.5065

## Calculate Entropy

### Whole Molecule level

In [18]:
result_wm = poseidon_object.run_analysis(level_list = ['moleculeLevel'], verbose=False, forceUnits="Kcal") # this is because the forces value supplied in this trajectory is in Kcal
print(result_wm)

2022-07-28 18:08:43.031855

solvent: ['WAT', 'wat', 'SOL', 'H2O', 'h2o', 'WAT_O', 'TIP3']

water: ['WAT', 'wat', 'SOL', 'H2O', 'h2o', 'WAT_O', 'TIP3']

1. Populate Dictionaries

memory use: 1.929 GB
0:00:00.001169
memory use: 1.929 GB
0:00:01.406683
0:00:01.407889
memory use: 1.929 GB

Total number of frames: 20.0
Number of atoms in each frame: 916
Number of variables in each list: 19

2. Process Dictionaries
['moleculeLevel']
---level: moleculeLevel


0:00:01.603452
{'moleculeLevel': {'solventData':    nearest assigned shell_num   variable      value count
0      ACE      WAT       1.0  Sor_test2   4.489916   155
1      ACE      WAT       1.0     Strans  47.114194   155
2      ACE      WAT       1.0       Srot  21.518819   155
3      ACE      WAT       1.0      count       7.75   155
4      ARG      WAT       1.0  Sor_test2   4.389286   358
5      ARG      WAT       1.0     Strans  46.659296   358
6      ARG      WAT       1.0       Srot  21.813924   358
7      ARG      WAT       1.0 

/home/donald/miniconda3/envs/CEtrial/lib/python3.10/site-packages/CodeEntropy/poseidon/analysis/EECalculation.py:859: RuntimeWarning: invalid value encountered in double_scalars
  w = w ** 0.5


### Residue Level

In [4]:
result_res = poseidon_object.run_analysis(level_list = ['residLevel_resname'], verbose=False, forceUnits="Kcal") # this is because the forces value supplied in this trajectory is in Kcal
print(result_res)

2022-06-30 14:03:08.891780

solvent: ['WAT', 'wat', 'SOL', 'H2O', 'h2o', 'WAT_O', 'TIP3']

water: ['WAT', 'wat', 'SOL', 'H2O', 'h2o', 'WAT_O', 'TIP3']

1. Populate Dictionaries

memory use: 0.232 GB
0:00:00.001426
memory use: 0.232 GB
0:00:01.297753
0:00:01.298211
memory use: 0.232 GB

Total number of frames: 20.0
Number of atoms in each frame: 916
Number of variables in each list: 19

2. Process Dictionaries
['residLevel_resname']
---level: residLevel_resname


0:00:01.458315
{'residLevel_resname': {'solventData':    nearest assigned shell_num   variable      value count
0    ACE_1      WAT       1.0  Sor_test2   4.489916   155
1    ACE_1      WAT       1.0     Strans  47.114194   155
2    ACE_1      WAT       1.0       Srot  21.518819   155
3    ACE_1      WAT       1.0      count       7.75   155
4    ARG_2      WAT       1.0  Sor_test2   4.389286   358
5    ARG_2      WAT       1.0     Strans  46.659296   358
6    ARG_2      WAT       1.0       Srot  21.813924   358
7    ARG_2     

/home/donald/miniconda3/envs/CEtrial/lib/python3.10/site-packages/CodeEntropy/poseidon/analysis/EECalculation.py:859: RuntimeWarning: invalid value encountered in double_scalars
  w = w ** 0.5


### United Atom Level

In [5]:
result_ua = poseidon_object.run_analysis(level_list = ['atomLevel'], verbose=False, forceUnits="Kcal") # this is because the forces value supplied in this trajectory is in Kcal
print(result_ua)

2022-06-30 14:03:13.104061

solvent: ['WAT', 'wat', 'SOL', 'H2O', 'h2o', 'WAT_O', 'TIP3']

water: ['WAT', 'wat', 'SOL', 'H2O', 'h2o', 'WAT_O', 'TIP3']

1. Populate Dictionaries

memory use: 0.232 GB
0:00:00.002788
memory use: 0.233 GB
0:00:01.388482
0:00:01.388816
memory use: 0.233 GB

Total number of frames: 20.0
Number of atoms in each frame: 916
Number of variables in each list: 19

2. Process Dictionaries
['atomLevel']
---level: atomLevel


0:00:01.590813
{'atomLevel': {'solventData':    nearest assigned shell_num   variable      value count
0    ACE_C    WAT_O       1.0  Sor_test2   3.859492    90
1    ACE_C    WAT_O       1.0     Strans   46.89642    90
2    ACE_C    WAT_O       1.0       Srot  21.477797    90
3    ACE_C    WAT_O       1.0      count        4.5    90
4    ACE_O    WAT_O       1.0  Sor_test2   1.383652    65
5    ACE_O    WAT_O       1.0     Strans  47.442017    65
6    ACE_O    WAT_O       1.0       Srot  21.607751    65
7    ACE_O    WAT_O       1.0      count  

/home/donald/miniconda3/envs/CEtrial/lib/python3.10/site-packages/CodeEntropy/poseidon/analysis/EECalculation.py:859: RuntimeWarning: invalid value encountered in double_scalars
  w = w ** 0.5


### Solute Contact

In [6]:
result_solcon = poseidon_object.run_analysis(level_list = ['soluteContacts'], verbose=False, forceUnits="Kcal") # this is because the forces value supplied in this trajectory is in Kcal
print(result_solcon)

2022-06-30 14:03:19.556689

solvent: ['WAT', 'wat', 'SOL', 'H2O', 'h2o', 'WAT_O', 'TIP3']

water: ['WAT', 'wat', 'SOL', 'H2O', 'h2o', 'WAT_O', 'TIP3']

1. Populate Dictionaries

memory use: 0.233 GB
0:00:00.001247
memory use: 0.233 GB
0:00:00.091347
0:00:00.091990
memory use: 0.233 GB

Total number of frames: 20.0
Number of atoms in each frame: 916
Number of variables in each list: 19

2. Process Dictionaries
['soluteContacts']
---level: soluteContacts


0:00:00.190073
{'soluteContacts': {'solventData':         nearest assigned shell_num   variable      value count
0   ACE_1_ACE_1      WAT       1.0  Sor_test2   0.747622    19
1   ACE_1_ACE_1      WAT       1.0     Strans  54.849501    19
2   ACE_1_ACE_1      WAT       1.0       Srot   21.83436    19
3   ACE_1_ACE_1      WAT       1.0      count       0.95    19
4   ACE_1_ARG_2      WAT       1.0  Sor_test2   0.129079    26
5   ACE_1_ARG_2      WAT       1.0     Strans  53.464565    26
6   ACE_1_ARG_2      WAT       1.0       Srot  21.